This section provides a concrete, narrative guide to implementing this analysis using the Python ecosystem. The code logic described here integrates Traja for kinematics, dlc2kinematics for velocity processing, and hmmlearn for segmentation.

### Step 1: Data Ingestion and Preprocessing
The analysis begins with the raw $(x,y)$ data, from DLC (BUT NOW WE ARE GETTING IT FROM SLEAP).
- Library: Traja.14
- Smoothing: Raw tracking data contains high-frequency jitter that can inflate velocity estimates. We apply a Savitzky-Golay filter.
- Code Concept: `trj.traja.smooth_sg(w=11, p=3)`. This fits a 3rd-order polynomial over an 11-frame window, preserving the sharp turns of VTE while removing noise.
- Rediscretization: Mice move at variable speeds. To analyze the shape of the path independent of time, we can rediscretize the trajectory into steps of constant length (e.g., 1 cm).
- Code Concept: `trj.traja.rediscretize(R=1.0)`. This is crucial for comparing the tortuosity of slow vs. fast trials.
### Step 2: Feature Extraction Logic
We calculate the feature matrix for the HMM.
1. Velocity and Acceleration (dlc2kinematics)
While Traja calculates speed, dlc2kinematics is optimized for DLC output and handles confidence scores well.22
- Logic: Compute velocity vectors for the snout and centroid. Use the centroid for locomotion state and the snout for head-scanning metrics.
2. Calculating IdPhi (Traja)
- Logic: Calculate the turn angle between consecutive triplets of points.
- angles = trj.traja.calc_turn_angle()
- idphi = angles.abs().rolling(window=30).sum()
- Nuance: A 1-second rolling window (approx 30 frames) is standard for capturing the "bout" of VTE.
### Generating the Nesting Metric (Morphology)
- Logic: If using standard DLC, calculate the distance between the "Nose" and "TailBase" points.
- body_length = distance(Nose, TailBase)
- During locomotion, body_length is maximal. During nesting/grooming, body_length decreases as the animal curls. Use the variance of this length over time as a feature.
### Step 3: HMM Segmentation (hmmlearn)
We utilize hmmlearn to perform the clustering.23
- Data Preparation: Stack the features (Log-Speed, IdPhi, Body-Length-Variance) into a matrix $X$.
- Normalization: HMMs (specifically Gaussian ones) are sensitive to scale. We must Z-score the data (mean=0, variance=1) using sklearn.preprocessing.StandardScaler.
#### Model Definition:
- model = hmm.GaussianHMM(n_components=3, covariance_type="full")
- Initialization Strategy: To ensure the model finds the "correct" 3 states, it is often helpful to initialize the means using K-Means clustering first.
#### Fitting and Prediction:
model.fit(X) learns the transition and emission matrices.
states = model.predict(X) assigns a label (0-2) to every frame.
### Step 4: Post-Hoc Validation and Ethogram Construction
The output of the HMM is a sequence of integers. We must map these to our ethological definitions.
- Spatial Validation: Plot the occupancy heatmap for each state.
- Nest State: Should cluster heavily in the Start Box or Corners.
- Exploit State: Should form a "highway" between the start and the correct reward port.
- Explore State: Should cover the maze edges (thigmotaxis) and the area immediately in front of the gratings (decision zone).
- Temporal Validation: Plot the state probability over the session duration.
We expect $P(S_{Nest})$ to increase linearly with time as satiety sets in. If $S_{Nest}$ is random, the model may be misidentifying "freezing" as nesting.


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from scipy.signal import savgol_filter, medfilt
from hmmlearn import hmm
from sklearn.preprocessing import StandardScaler
import ipywidgets as widgets

# ==========================================
# 1. CONFIGURATION
# ==========================================
DLC_FILE = "C:/Users/aleja/Box/Awake Project/Maze data/simplermaze/mouse 6357/deeplabcut/mouse6357/mouse6357-shahd-2025-09-08/videos/6357_2024-08-28_11_58_14s3.6DLC_Resnet50_mouse6357Sep8shuffle1_snapshot_200.csv"

TRIAL_FILE = "C:/Users/aleja/Box/Awake Project/Maze data/simplermaze/mouse 6357/2024-08-28_11_58_146357session3.6/trials_corrected_final_frames.csv"

# Cleaning Parameters
LIKELIHOOD_THRESH = 0.5  # Drop points with confidence below 10%
MEDIAN_WINDOW = 5        # Remove single-frame jumps
SMOOTH_WINDOW = 31       # Stronger smoothing for velocity (approx 1 sec)
FPS = 30

# ==========================================
# 2. DATA LOADING & CLEANING
# ==========================================
def load_and_clean_data():
    print("1. Loading Data...")
    df_dlc = pd.read_csv(DLC_FILE, header=[0, 1, 2], index_col=0)
    scorer = df_dlc.columns.get_level_values(0)[0]
    
    data = pd.DataFrame()
    
    # Process each bodypart
    print(f"2. Cleaning Data (Threshold p < {LIKELIHOOD_THRESH})...")
    for bp in ['nose', 'mid', 'tailbase']:
        # Extract raw columns
        x = df_dlc[scorer][bp]['x']
        y = df_dlc[scorer][bp]['y']
        p = df_dlc[scorer][bp]['likelihood']
        
        # FILTER: Set low confidence points to NaN
        bad_points = p < LIKELIHOOD_THRESH
        x[bad_points] = np.nan
        y[bad_points] = np.nan
        
        # INTERPOLATE: Fill gaps linearly
        x = x.interpolate(method='linear', limit_direction='both')
        y = y.interpolate(method='linear', limit_direction='both')
        
        # SMOOTH: Median filter removes sharp "teleporting" jumps
        x = medfilt(x, kernel_size=MEDIAN_WINDOW)
        y = medfilt(y, kernel_size=MEDIAN_WINDOW)
        
        data[f'{bp}_x'] = x
        data[f'{bp}_y'] = y
        data[f'{bp}_p'] = p  # Keep raw p for reference

    # Load Trials
    df_trials = pd.read_csv(TRIAL_FILE)
    
    return data, df_trials

df_tracking, df_trials = load_and_clean_data()
print("   Data cleaned and ready.")

# ==========================================
# 3. FEATURE ENGINEERING (ROBUST)
# ==========================================
print("3. Extracting Features...")

# A. Velocity (Smoothed)
dx = np.diff(df_tracking['mid_x'], prepend=df_tracking['mid_x'][0])
dy = np.diff(df_tracking['mid_y'], prepend=df_tracking['mid_y'][0])
raw_vel = np.sqrt(dx**2 + dy**2) * FPS
# Heavy smoothing to fix "jitter" noise
df_tracking['velocity'] = savgol_filter(raw_vel, window_length=SMOOTH_WINDOW, polyorder=3)
df_tracking['log_velocity'] = np.log(df_tracking['velocity'] + 1e-6)

# B. IdPhi (Heading Changes)
# Note: Nose tracking is poor (12% good), so this feature might still be noisy.
nose_dx = np.diff(df_tracking['nose_x'], prepend=df_tracking['nose_x'][0])
nose_dy = np.diff(df_tracking['nose_y'], prepend=df_tracking['nose_y'][0])
heading = np.degrees(np.arctan2(nose_dy, nose_dx))
dphi = np.diff(heading, prepend=heading[0])
dphi = (dphi + 180) % 360 - 180 
# Stronger rolling sum to smooth out heading jitter
df_tracking['idphi'] = pd.Series(np.abs(dphi)).rolling(window=SMOOTH_WINDOW, center=True).sum().fillna(0)

# C. Body Length
df_tracking['body_length'] = np.sqrt(
    (df_tracking['nose_x'] - df_tracking['tailbase_x'])**2 + 
    (df_tracking['nose_y'] - df_tracking['tailbase_y'])**2
)
# Smooth body length too
df_tracking['body_length'] = savgol_filter(df_tracking['body_length'], window_length=SMOOTH_WINDOW, polyorder=3)

# ==========================================
# 4. HMM SEGMENTATION
# ==========================================
print("4. Training Model on Valid Trials...")

# Tag Valid Frames
valid_mask = np.zeros(len(df_tracking), dtype=bool)
for _, row in df_trials.iterrows():
    start, end = int(row['start_frame']), int(row['end_frame'])
    if start < len(df_tracking) and end < len(df_tracking):
        valid_mask[start:end] = True

# Prepare Data
features = ['log_velocity', 'idphi', 'body_length']
X = df_tracking[features].values
X_train = df_tracking.loc[valid_mask, features].values
X_train = np.nan_to_num(X_train)

# Scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Fit HMM
model = hmm.GaussianHMM(n_components=4, covariance_type="full", n_iter=100, random_state=42)
model.fit(X_train_scaled)

# Predict
X_all_scaled = scaler.transform(np.nan_to_num(X))
df_tracking['state_hmm'] = model.predict(X_all_scaled)

# ==========================================
# 5. AUTOMATIC STATE LABELING
# ==========================================
# Map states 0-3 to Explore(1), Exploit(2), Nest(3), Groom(4)
means = pd.DataFrame(model.means_, columns=features)
print("\nState Cluster Means:")
print(means)

# 1. Exploit = Highest Velocity
exploit_cluster = means['log_velocity'].idxmax()
# 2. Nest = Lowest Velocity
nest_cluster = means['log_velocity'].idxmin()
# 3. Explore vs Groom
remaining = list(set(range(4)) - {exploit_cluster, nest_cluster})
# Explore has higher velocity than Groom
if means.loc[remaining[0], 'log_velocity'] > means.loc[remaining[1], 'log_velocity']:
    explore_cluster, groom_cluster = remaining[0], remaining[1]
else:
    explore_cluster, groom_cluster = remaining[1], remaining[0]

state_map = {explore_cluster: 'Explore (1)', exploit_cluster: 'Exploit (2)', 
             nest_cluster: 'Nest (3)', groom_cluster: 'Groom (4)'}
state_id_map = {explore_cluster: 1, exploit_cluster: 2, nest_cluster: 3, groom_cluster: 4}

df_tracking['state_name'] = df_tracking['state_hmm'].map(state_map)
df_tracking['state_id'] = df_tracking['state_hmm'].map(state_id_map)
print(f"Mapped States: {state_map}")

# ==========================================
# 6. INTERACTIVE PLOT
# ==========================================
def plot_trial(trial_id):
    row = df_trials[df_trials['matched_trial_id'] == trial_id].iloc[0]
    start, end = int(row['start_frame']), int(row['end_frame'])
    
    # Get Data Slice
    df_slice = df_tracking.loc[start:end].copy()
    
    # Calculate Sequences
    pred_seq = ",".join(df_slice['state_id'].loc[df_slice['state_id'].shift() != df_slice['state_id']].astype(str))
    print(f"Trial: {trial_id}")
    print(f"Manual Sequence: {row.get('state_sequence', 'N/A')}")
    print(f"Predicted Seq:   {pred_seq}")

    # Plot
    colors = {'Explore (1)': '#ff7f0e', 'Exploit (2)': '#2ca02c', 'Nest (3)': '#1f77b4', 'Groom (4)': '#9467bd'}
    
    fig = px.scatter(df_slice, x='mid_x', y='mid_y', color='state_name', 
                     color_discrete_map=colors, title=f"Cleaned Trajectory: {trial_id}",
                     labels={'mid_x':'X', 'mid_y':'Y'}, height=500, width=600)
    fig.update_yaxes(autorange="reversed")
    fig.update_layout(yaxis_scaleanchor="x")
    fig.update_traces(marker=dict(size=4)) # Smaller markers for cleaner look
    fig.show()

# Dropdown
trial_list = df_trials['matched_trial_id'].unique().tolist()
widgets.interact(plot_trial, trial_id=widgets.Dropdown(options=trial_list));

1. Loading Data...
2. Cleaning Data (Threshold p < 0.5)...
   Data cleaned and ready.
3. Extracting Features...
4. Training Model on Valid Trials...


c:\Users\aleja\anaconda3\envs\malawi\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning:

invalid value encountered in log




State Cluster Means:
   log_velocity     idphi  body_length
0     -0.678785 -0.826449    -0.260792
1      0.972168  0.740301     0.494217
2      0.076137  1.626100     0.223462
3      0.814647 -0.223506    -0.081838
Mapped States: {3: 'Explore (1)', 1: 'Exploit (2)', 0: 'Nest (3)', 2: 'Groom (4)'}


interactive(children=(Dropdown(description='trial_id', options=('trial_000', 'trial_001', 'trial_002', 'trial_…

In [4]:
%pip install hmmlearn

Note: you may need to restart the kernel to use updated packages.
